## Google Colab Set Up

The code cells below link this notebook to Google Colab. If you are running this notebook in a Jupyter Environment, there may be some packages you will need to install. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd "/content/drive/My Drive/Research/GAN/CycleGAN"

/content/drive/My Drive/Research/GAN/CycleGAN


In [3]:
%tensorflow_version 1.x
!pip install scipy==1.1.0
import scipy
print(scipy.__version__)
import tensorflow
print(tensorflow.__version__)

TensorFlow 1.x selected.
1.1.0
1.15.2


##Environment Set Up

The code cells below create relevant files for the environment. 

The code cell below downloads a dataset using one of the original datasets from CycleGAN

In [0]:
!bash ./download_datasets.sh summer2winter_yosemite

mkdir: cannot create directory ‘./input’: File exists
for details.

--2020-05-28 15:09:34--  https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/summer2winter_yosemite.zip
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.189.73
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.189.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 132640856 (126M) [application/zip]
Saving to: ‘./input/summer2winter_yosemite.zip’

./input/summer2wint 100%[===================>] 126.50M  17.5MB/s    in 8.5s    

2020-05-28 15:09:43 (14.9 MB/s) - ‘./input/summer2winter_yosemite.zip’ saved [132640856/132640856]

Archive:  ./input/summer2winter_yosemite.zip
   creating: ./input/summer2winter_yosemite/trainA/
  inflating: ./input/summer2winter_yosemite/trainA/2014-07-01 10:31:48.jpg  
  inflating: ./input/summer2winter_yosemite/trainA/2015-07-01 23:43:46.jpg  
  inflating: ./input/summer2winter_yosemite/trainA/2014-07-27 21:5

The first code cell outputs the sizes of the train and test datasets. This information is used for the variables in the following code cell. Be sure to add your datasets to the dictionaries below. This information is needed to create labels for your data.

In [0]:
!ls -l input/summer2winter_yosemite/trainB | wc -l
!ls -l input/summer2winter_yosemite/trainA | wc -l
!ls -l input/summer2winter_yosemite/testB | wc -l
!ls -l input/summer2winter_yosemite/testA | wc -l

963
1232
239
310


In [0]:
DATASET_TO_SIZES = {
    'facades_train': 401,
    'facades_test': 107,
    'summer2winter_yosemite_train': 1232,
    'summer2winter_yosemite_test': 310
}

DATASET_TO_IMAGETYPE = {
    'facades_train': '.jpg',
    'facades_test': '.jpg',
    'summer2winter_yosemite_train': '.jpg',
    'summer2winter_yosemite_test': '.jpg'
}

PATH_TO_CSV = {
    'facades_train': './input/facades/facades_train.csv',
    'facades_test': './input/facades/facades_test.csv',
    'summer2winter_yosemite_train': './input/summer2winter_yosemite/summer2winter_yosemite_train.csv',
    'summer2winter_yosemite_test': './input/summer2winter_yosemite/summer2winter_yosemite_test.csv',
}

with open("cyclegan_datasets.py", "w") as outfile:
    outfile.write('"""Contains the standard train/test splits for the cyclegan data."""\n')
    outfile.write('"""The size of each dataset. Usually it is the maximum number of images from each domain."""\n')
    
    s = '\n'.join(["'%s': %d," % (k,DATASET_TO_SIZES[k]) for k in DATASET_TO_SIZES])
    outfile.write("DATASET_TO_SIZES = {\n%s}\n" % (s))

    s = '\n'.join(["'%s': '%s'," % (k,DATASET_TO_IMAGETYPE[k]) for k in DATASET_TO_IMAGETYPE])
    outfile.write("DATASET_TO_IMAGETYPE = {\n%s}\n" % (s))

    s = '\n'.join(["'%s': '%s'," % (k,PATH_TO_CSV[k]) for k in PATH_TO_CSV])
    outfile.write("PATH_TO_CSV = {\n%s}\n" % (s))

    outfile.close()

import cyclegan_datasets
import importlib
importlib.reload(cyclegan_datasets)

assert(cyclegan_datasets.DATASET_TO_SIZES == DATASET_TO_SIZES)
assert(cyclegan_datasets.DATASET_TO_IMAGETYPE == DATASET_TO_IMAGETYPE)
assert(cyclegan_datasets.PATH_TO_CSV == PATH_TO_CSV)

The code cell below creates a csv file for the entire dataset.

In [0]:
import create_cyclegan_dataset
import importlib
importlib.reload(create_cyclegan_dataset)
from create_cyclegan_dataset import create_dataset


name = 'summer2winter_yosemite'
typ = 'train'
image_path_a = 'input/%s/%sA' % (name, typ)
image_path_b = 'input/%s/%sB' % (name, typ)
dataset_name = '%s_%s' % (name, typ)
do_shuffle = 0
create_dataset(image_path_a, image_path_b, dataset_name, do_shuffle, DATASET_TO_SIZES, DATASET_TO_IMAGETYPE, PATH_TO_CSV)

## Training

The code block below trains the model. Set to_train to 1 if it is your first time training this model. Set to_train to 2 if you are using a previous model to train. Previous models can be found in output/cyclegan/your_dataset/. Specify training configurations in the json files below. 

If you want to create a new json file for your dataset, replace the dataset_name and json file in the code cell below.

In [0]:
!rm configs/summer2winter_yosemite_test.json
!echo '{ "description": "Testing with trained model.", "network_version": "tensorflow", "dataset_name": "summer2winter_yosemite_test", "do_flipping": 0}' >> configs/summer2winter_yosemite_test.json
!rm configs/summer2winter_yosemite.json
!echo '{ "description": "The Tensorflow version of CycleGAN.", "pool_size": 50, "base_lr":0.0002, "max_step": 200, "network_version": "tensorflow", "dataset_name": "summer2winter_yosemite_train", "do_flipping": 1, "_LAMBDA_A": 10, "_LAMBDA_B": 10 }' >> configs/summer2winter_yosemite.json

In [5]:
!ls "output/cyclegan/summer2winter_yosemite/"

20200528-154528  20200528-170433  20200607-154800  20200608-132900
20200528-164522  20200529-041955  20200607-160714  20200608-134717


In [6]:
import importlib
import data_loader
importlib.reload(data_loader)
from main import main

to_train = 2
name = 'summer2winter_yosemite'
typ = "20200608-134717"
log_dir = "output/cyclegan/%s" % name
config_filename = "configs/%s.json" % name
skip = True
checkpoint_dir= "output/cyclegan/%s/%s" % (name, typ)
main(to_train, log_dir, config_filename, checkpoint_dir, skip, save_training_images=False)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.

100%|██████████| 1232/1232 [09:05<00:00,  2.26it/s]

Epoch 111/200


100%|██████████| 1232/1232 [04:53<00:00,  4.20it/s]


Epoch 112/200


...

100%|██████████| 1232/1232 [04:48<00:00,  4.28it/s]


Epoch 198/200


100%|██████████| 1232/1232 [04:48<00:00,  4.28it/s]


Epoch 199/200


100%|██████████| 1232/1232 [04:48<00:00,  4.28it/s]


##Testing

The code cell below tests a model with the test dataset. Previous models can be found in output/cyclegan/your_dataset/. The results will be saved at the same path (with a new timestamp for the test). 

In [0]:
!ls "output/cyclegan/summer2winter_yosemite/"

In [0]:
 from main import main

name = 'summer2winter_yosemite'
ckpt = '20200607-160714'
to_train = 0
log_dir = "output/cyclegan/%s" % name
config_filename = "configs/%s_test.json" % name
skip = True
checkpoint_dir= "output/cyclegan/%s/%s" % (name, ckpt)
main(to_train, log_dir, config_filename, checkpoint_dir, skip)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Testing the results
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. U

  0%|          | 0/310 [00:00<?, ?it/s]

Saving Images


100%|██████████| 310/310 [03:13<00:00,  1.61it/s]


## Save Images For Analysis

In [0]:
!ls output/cyclegan/summer2winter_yosemite/20200608-132900/imgs/inputA*

output/cyclegan/summer2winter_yosemite/20200608-132900/imgs/inputA_0_0.jpg
output/cyclegan/summer2winter_yosemite/20200608-132900/imgs/inputA_0_100.jpg
output/cyclegan/summer2winter_yosemite/20200608-132900/imgs/inputA_0_101.jpg
output/cyclegan/summer2winter_yosemite/20200608-132900/imgs/inputA_0_102.jpg
output/cyclegan/summer2winter_yosemite/20200608-132900/imgs/inputA_0_103.jpg
output/cyclegan/summer2winter_yosemite/20200608-132900/imgs/inputA_0_104.jpg
output/cyclegan/summer2winter_yosemite/20200608-132900/imgs/inputA_0_105.jpg
output/cyclegan/summer2winter_yosemite/20200608-132900/imgs/inputA_0_106.jpg
output/cyclegan/summer2winter_yosemite/20200608-132900/imgs/inputA_0_107.jpg
output/cyclegan/summer2winter_yosemite/20200608-132900/imgs/inputA_0_108.jpg
output/cyclegan/summer2winter_yosemite/20200608-132900/imgs/inputA_0_109.jpg
output/cyclegan/summer2winter_yosemite/20200608-132900/imgs/inputA_0_10.jpg
output/cyclegan/summer2winter_yosemite/20200608-132900/imgs/inputA_0_110.jpg
ou

In [0]:
!mkdir -p "/content/drive/My Drive/Research/GAN/results/CycleGAN/summer_real"
!mkdir -p "/content/drive/My Drive/Research/GAN/results/CycleGAN/summer_fake"
!mkdir -p "/content/drive/My Drive/Research/GAN/results/CycleGAN/winter_real"
!mkdir -p "/content/drive/My Drive/Research/GAN/results/CycleGAN/winter_fake"

!cp output/cyclegan/summer2winter_yosemite/20200608-132900/imgs/inputA* "/content/drive/My Drive/Research/GAN/results/CycleGAN/summer_real/"
!cp output/cyclegan/summer2winter_yosemite/20200608-132900/imgs/inputB* "/content/drive/My Drive/Research/GAN/results/CycleGAN/winter_real/"
!cp output/cyclegan/summer2winter_yosemite/20200608-132900/imgs/fakeA* "/content/drive/My Drive/Research/GAN/results/CycleGAN/winter_fake/"
!cp output/cyclegan/summer2winter_yosemite/20200608-132900/imgs/fakeB* "/content/drive/My Drive/Research/GAN/results/CycleGAN/summer_fake/"